In [ ]:
# define rooms and items

couch = {
    "name": "couch",
    "type": "furniture",
}

double_bed = {
    "name": "double bed",
    "type": "furniture",
}

dresser = {
    "name": "dresser",
    "type": "furniture",
}

door_a = {
    "name": "door a",
    "type": "door",
}

door_b = {
    "name": "door b",
    "type": "door",
}

door_c = {
    "name": "door c",
    "type": "door",
}

door_d = {
    "name": "door d",
    "type": "door",
}


queen_bed = {
    "name": "queen bed",
    "type": "furniture",
}


key_a = {
    "name": "key for door a",
    "type": "key",
    "target": door_a,
}

key_b = {
    "name": "key for door b",
    "type": "key",
    "target": door_b,
}

key_c = {
    "name": "key for door c",
    "type": "key",
    "target": door_c,
}

key_d = {
     "name": "key for door d",
    "type": "key",
    "target": door_d,
}

piano = {
    "name": "piano",
    "type": "furniture",
}
painting = {
    "name": "painting",
    "type": "furniture",
}

dining_table = {
    "name": "dining table",
    "type": "furniture",
}

game_room = {
    "name": "game room",
    "type": "room",
}

bedroom_1 = {
    'name':"bedroom 1",
    'type':'room'
}

bedroom_2 = {
    'name':"bedroom 2",
    'type':'room'
}

living_room = {
    'name':"living room",
    'type':'room'
}

outside = {
  "name": "outside"
}
safe = {
    "name": "safe",
    "type": "safe",
}
hidden_note = {
    "name": "note",
    "type": "note",
}

all_rooms = [game_room, outside, bedroom_1, bedroom_2, living_room]

all_doors = [door_a, door_b, door_c, door_d]

# define which items/rooms are related

object_relations = {
    "game room": [couch, piano, door_a],
    "piano": [key_a],
    "outside": [door_a],
    "bedroom 1": [door_a, door_c, queen_bed, door_b, painting],
    "door a": [game_room, bedroom_1],
    "door b": [bedroom_1, bedroom_2],
    "door c": [bedroom_1, living_room],
    "queen bed": [key_b],
    "bedroom 2": [double_bed, dresser, safe, door_b],
    "dresser": [key_d],
    "double bed": [],
    "living room": [door_d, dining_table],
    "door d": [living_room, outside],
    "painting": [hidden_note],
    "safe": [key_c],
}

# define game state. Do not directly change this dict. 
# Instead, when a new game starts, make a copy of this
# dict and use the copy to store gameplay state. This 
# way you can replay the game multiple times.

INIT_GAME_STATE = {
    "current_room": game_room,
    "inventory": [],
    "target_room": outside
}

In [ ]:
def linebreak():
    """
    Print a line break to separate game events visually.
    """
    print("\n\n")

def start_game():
    """
    Start the game. 
    Sets the initial context and intro story, then enters the main game loop.
    """
    print("You wake up on a couch and find yourself in a strange house with no windows which you have never been to before. You don't remember why you are here and what had happened before. You feel some unknown danger is approaching and you must get out of the house, NOW!")
    play_room(game_state["current_room"])

def play_room(room):
    """
    The main game loop logic for a specific room.
    1. Checks if the player has reached the target room (Victory Condition).
    2. If not, prompts the player for an action (explore, examine, inventory).
    """
    game_state["current_room"] = room
    
    # --- VICTORY CHECK ---
    if(game_state["current_room"] == game_state["target_room"]):
        print("Congrats! You escaped the room!")
    else:
        # --- MAIN ACTION LOOP ---
        print("You are now in " + room["name"])
        intended_action = input("What would you like to do? Type 'explore' or 'examine' or 'inventory'?").lower().strip()
        
        if intended_action == "explore":
            explore_room(room)
            play_room(room) # Reloads the room after exploring
            
        elif intended_action == "examine":
            # Passes user input directly to the examine function
            examine_item(input("What would you like to examine?").lower().strip())
            
        elif intended_action == "inventory":
            check_inventory()
            play_room(room) # Reloads room after checking pockets
            
        else:
            print("Not sure what you mean. Type 'explore' or 'examine'.")
            play_room(room)
        
        linebreak()

def check_inventory():
    """
    Lists all items currently held by the player.
    """
    if len(game_state['inventory']) == 0:
        print('You have no keys in your inventory')
    else:
        output_message = 'You check your pockets and find the keys'
        print(output_message)
        for key in game_state['inventory']:
            print(key['name'])

def explore_room(room):
    """
    Explore a room. List all items belonging to this room.
    Uses a loop to build a comma-separated string of items.
    """
    explore_message = "You explore the room. This is " + room["name"] + ". You find "
    for item in object_relations[room["name"]]:
        explore_message += str(item["name"]) + ", "
    
    # Removes the last comma and space, adds a period.
    explore_message = explore_message[:-2] + "." 
    print(explore_message)

def get_next_room_of_door(door, current_room):
    """
    From object_relations, find the two rooms connected to the given door.
    Return the second room (the one we are not currently in).
    """
    connected_rooms = object_relations[door["name"]]
    # Logic assumption: The door connects exactly two rooms.
    # TODO: Add logic to ensure we don't return the room we are currently in.
    return connected_rooms[1]

def examine_item(item_name):
    """
    The core interaction function.
    1. Compiles a list of everything the player can see (Room + Inventory).
    2. Loops through to find the matching item.
    3. Executes specific logic based on item type (Door, Safe, Note, Generic).
    """
    
    current_room = game_state["current_room"]
    
    # Combine room items and inventory so player can examine things in their pocket
    items_to_check = object_relations[current_room["name"]] + game_state["inventory"]
    
    next_room = ""
    output = None
    
    for item in items_to_check:
        if(item["name"] == item_name):
            output = "You examine " + item_name + ". "
            
            # --- DOOR LOGIC ---
            if(item["type"] == "door"):
                have_key = False
                for key in game_state["inventory"]:
                    # BUG FIX: Added "target" in key check to prevent crashing on non-key items (like notes)
                    if("target" in key and key["target"] == item):
                        have_key = True
                
                if(have_key):
                    output += "You unlock it with a key you have."
                    next_room = get_next_room_of_door(item, current_room)
                else:
                    output += "It is locked but you don't have the key."
            
            # --- SAFE LOGIC ---
            elif item["type"] == "safe":
                output += "You examine the safe. It's a combination lock."
                code = input("Enter the combination to open the safe:")
                if code == "6596":
                    output += "The safe is now open."
                    # Check if safe has hidden items inside
                    if(item["name"] in object_relations and len(object_relations[item["name"]])>0):
                        item_found = object_relations[item["name"]].pop()
                        game_state["inventory"].append(item_found)
                        output += "You find " + item_found["name"] + "."
                else:
                    output += "The combination is incorrect."
            
            # --- NOTE LOGIC ---
            elif item["type"] == "note":
                output += "The note reads: 'I am the lists rigid sibling.\nYou can see what I hold,\nbut you can never change my mind.\n Wrap me in parentheses, and I'll keep your secrets forever\n"
                output += "Dictionary : 2076\n"
                output += "Tuple : 6596\n"
                output += "Set : 3448\n"    
            
            # --- GENERIC ITEM LOGIC (Hidden Objects) ---
            # If it's not a door, safe, or note, check if it hides something (like the piano or painting)
            elif (item["name"] in object_relations and len(object_relations[item["name"]])>0):
                item_found = object_relations[item["name"]].pop()
                game_state["inventory"].append(item_found)
                output += "You find " + item_found["name"] + "."
            
            # --- NOTHING INTERESTING ---
            else:
                output += "There isn't anything interesting about it."
            
            # Print the result and exit loop once item is found
            print(output)
            break

    if(output is None):
        print("The item you requested is not found in the current room.")

    # --- ROOM TRANSITION ---
    # Only asks to move if a next_room was set (i.e., a door was unlocked)
    if(next_room and input("Do you want to go to the next room? enter 'yes' or 'no'").lower().strip() == 'yes'):
        play_room(next_room)
    else:
        play_room(current_room)

In [ ]:
game_state = INIT_GAME_STATE.copy()

start_game()

You wake up on a couch and find yourself in a strange house with no windows which you have never been to before. You don't remember why you are here and what had happened before. You feel some unknown danger is approaching and you must get out of the house, NOW!
You are now in game room
What would you like to do? Type 'explore' or 'examine'?explore
You explore the room. This is game room. You find couch, piano, door a
You are now in game room
What would you like to do? Type 'explore' or 'examine'?examine
What would you like to examine?door a
You examine door a. It is locked but you don't have the key.
You are now in game room
What would you like to do? Type 'explore' or 'examine'?examine
What would you like to examine?piano
You examine piano. You find key for door a.
You are now in game room
What would you like to do? Type 'explore' or 'examine'?examine
What would you like to examine?door a
You examine door a. You unlock it with a key you have.
Do you want to go to the next room? Ener 